<a href="https://colab.research.google.com/github/bhnunes/CNNStudy/blob/main/CNNStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import json
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array, load_img
from keras.applications.imagenet_utils import preprocess_input
import requests

In [17]:
# URL of the raw file on GitHub
url = "https://raw.githubusercontent.com/bhnunes/CNNStudy/main/imagenet_class_index.json"
testUrl = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/German_Shepherd_-_DSC_0346_%2810096362833%29.jpg/220px-German_Shepherd_-_DSC_0346_%2810096362833%29.jpg"


def getFile(url,fileName, mode):
  response = requests.get(url)
  if response.status_code ==  200:
      with open(fileName, mode) as file:
        if mode =='w':
          file.write(response.text)
        else:
          file.write(response.content)
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")





In [18]:
getFile(url,'imagenet_class_index.json', 'w')
getFile(testUrl,'test_image.jpg', 'wb')


In [19]:
#download VGG16
vgg16=VGG16(weights='imagenet')

In [20]:
vgg16.save('vgg16.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
def prepare_image(img_path):
  img=load_img(img_path, target_size=(224,224))
  x=img_to_array(img)
  x=np.expand_dims(x,axis=0)
  x=preprocess_input(x)
  return x

In [22]:
imagePath = "/content/test_image.jpg"
y = prepare_image(imagePath)

In [23]:
preds=vgg16.predict(y)

1/1 [==============================] - 1s 1s/step


In [24]:
CLASS_INDEX=None
CLASS_INDEX_PATH="/content/imagenet_class_index.json"

In [25]:
def get_predictions(preds,top=5):
  global CLASS_INDEX

  CLASS_INDEX=json.load(open(CLASS_INDEX_PATH))

  results=[]
  for pred in preds:
    top_indices=pred.argsort()[-top:][::-1]
    result=[tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
    result.sort(key=lambda x: x[2], reverse=True)
    results.append(result)
  return results

In [26]:
print(get_predictions(preds,top=5))

[[('n02106662', 'German_shepherd', 0.99837816), ('n02105162', 'malinois', 0.0015710895), ('n03803284', 'muzzle', 1.9646503e-05), ('n02105056', 'groenendael', 9.4065235e-06), ('n02105412', 'kelpie', 7.3132423e-06)]]
